# Interactive Dashboard Using Plotly Dash

Install wget, dash and import Required Libraries

In [ ]:
# Install wget and dash

!pip3 install wget dash

In [ ]:
# Import required libraries

import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

Read launch data csv file

In [ ]:
#Read launch data csv file

spacex_launch_file = wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")


Calculate maximum and minimum payload

In [ ]:
#Calculate maximum and minimum payload

max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

Create a dash application

In [ ]:
# Create a dash application

app = dash.Dash(__name__)

Create an app layout

In [ ]:
#Create an app layout

app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
 
                                dcc.Dropdown(id='site-dropdown',
                                    options=[
                                        {'label':'All','value':'All'},
                                        {'label':'CCAFS LC-40','value':'CCAFS LC-40'},
                                        {'label':'CCAFS SLC-40','value':'CCAFS SLC-40'}, 
                                        {'label':'KSC LC-39A','value':'KSC LC-39A'},
                                        {'label':'VAFB SLC-4E','value':'VAFB SLC-4E'},
                                        ],
                                    value='All',
                                    placeholder='Select a Launch Site',
                                    searchable=True,
                                    ),
                                html.Br(),

                                
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
 
                                dcc.RangeSlider(id='payload-slider',
                                    min=0, max=10000, step=1000,
                                    marks={0:'0 Kg',1000:'1000 Kg',2000:'2000 Kg',3000:'3000 Kg',
                                           4000:'4000 Kg', 5000:'5000 Kg', 6000:'6000 Kg', 7000:'7000 Kg',
                                           8000:'8000 Kg', 9000:'9000 Kg', 10000:'10000 Kg'},
                                    value=[min_payload, max_payload]
                                ),

                               
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])


Callback for Pie Chart

In [ ]:
#Callback for Pie Chart (Select All or Specific Launch Site)

@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
               Input(component_id='site-dropdown', component_property='value'))

def get_graph(site):
    # Selected Site or All Sites
    if site == 'All':
        fig = px.pie(spacex_df, values='class', names='Launch Site', title='Total Successful Launches by Site')
    else:
        #select_site = []
        select_site = spacex_df[spacex_df['Launch Site'] == site ]
        select_site = select_site.groupby('class', as_index=False, sort=True).count() 
        fig = px.pie(select_site, values='Launch Site', names='class', 
            title="Failure('0') vs Success('1') Launches for site "+site,
            color="class",color_discrete_map={0:"red",1:"green"},
            #category_orders={"class":[0,1]},
            )

    return fig

Callback for Payload Scatter Chart

In [ ]:
#Callback for Payload Scatter Chart (Select payload range from slider bar)

@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
               [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id='payload-slider', component_property='value')])

def get_scatter(site,payloadRange):
    # Selected Site or All Sites
    
    low, high = payloadRange
    print("Site: ",site, "payloadRange: ",payloadRange, "low: ",low, "high: ",high)
    
    mask = (spacex_df['Payload Mass (kg)'] >= low) & (spacex_df['Payload Mass (kg)'] <= high)

    if site == 'All':
        fig = px.scatter(spacex_df[mask], x='Payload Mass (kg)', y='class', color="Booster Version Category")
    else:
        #select_site = []
        select_site = spacex_df[spacex_df['Launch Site'] == site ]
        fig = px.scatter(select_site[mask], x='Payload Mass (kg)', y='class', color="Booster Version Category")

    return fig

Run the app

In [ ]:
# Run the app

if __name__ == '__main__':
    app.run_server()